In [106]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import random
import nltk.classify.util
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [107]:
useless_words = stopwords.words("english")

In [108]:
movie_reviews.categories()

['neg', 'pos']

In [109]:
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append([movie_reviews.words(fileid),category])

#documents[0]
len(documents)

2000

In [110]:
documents(1200)

TypeError: 'list' object is not callable

In [111]:
documents[1200]

[['a', 'common', 'complaint', 'amongst', 'film', ...], 'pos']

In [112]:
random.shuffle(documents)

In [113]:
documents[1200]

[['one', 'of', 'the', 'best', 'movies', 'i', "'", 've', ...], 'pos']

In [114]:
useless_words1 = [',','.','-','_','(',')',':','?',';','"']
words = []
for word in movie_reviews.words():
    if word not in useless_words and word not in useless_words1:
        words.append(word.lower())
    
len(words)

746698

In [115]:
words_Freq=nltk.FreqDist(words)

In [116]:
words_Freq.most_common(10)

[("'", 30585),
 ('film', 9517),
 ('one', 5852),
 ('movie', 5771),
 ('like', 3690),
 ('even', 2565),
 ('good', 2411),
 ('time', 2411),
 ('story', 2169),
 ('would', 2109)]

In [117]:
words_Freq['excellent']

184

In [118]:
word_features = list(words_Freq.keys())[:8000]

In [119]:
featuresets = []
for (review,category) in documents:
    words = set([word for word in review if word not in useless_words and word not in useless_words1])
    features = {}
    
    for w in word_features:
        features[w] = w in words
    featuresets.append([features,category])
len(featuresets)

2000

In [128]:
training_set = featuresets[:1400]
testing_set = featuresets[1400:]

In [129]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [130]:
nltk.classify.accuracy(classifier, testing_set)

0.8066666666666666

In [131]:
classifier.show_most_informative_features(10)

Most Informative Features
                  sloppy = True              neg : pos    =     14.9 : 1.0
               stupidity = True              neg : pos    =     14.2 : 1.0
                  seagal = True              neg : pos    =     12.2 : 1.0
               ludicrous = True              neg : pos    =     11.3 : 1.0
           unintentional = True              neg : pos    =     10.2 : 1.0
                    3000 = True              neg : pos    =      9.6 : 1.0
                 idiotic = True              neg : pos    =      9.3 : 1.0
               traveling = True              pos : neg    =      9.1 : 1.0
                  hudson = True              neg : pos    =      8.9 : 1.0
               atrocious = True              neg : pos    =      8.3 : 1.0


In [132]:
import numpy as np
from sklearn.model_selection import KFold

In [133]:
kf = KFold(n_splits=10)
sum = 0
for train, test in kf.split(featuresets):
    train_data = np.array(featuresets)[train]
    test_data = np.array(featuresets)[test]
    classifier = nltk.NaiveBayesClassifier.train(train_data)
    sum += nltk.classify.accuracy(classifier, test_data)
average = sum/10

In [134]:
print average

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(average)? (<ipython-input-134-7735039faa9b>, line 1)

In [135]:
print(average)

0.819


In [136]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC

In [137]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
nltk.classify.accuracy(MNB_classifier, testing_set)

0.81

In [138]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
nltk.classify.accuracy(SVC_classifier,testing_set)

0.495

In [139]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
nltk.classify.accuracy(LinearSVC_classifier,testing_set)

0.8216666666666667

In [141]:
try_rev = '''Amazing piece of art. Worldclass. Awesome. Worth it.A bit shady.'''

In [142]:
try_word = word_tokenize(try_rev)

In [143]:
def create_word_features(words):
    useful_words = [word for word in words if word not in useless_words and word not in useless_words1]
    my_dictionary = dict([(word, True) for word in useful_words])
    return my_dictionary
    

In [144]:
try_word = create_word_features(try_word)

In [145]:
classifier.classify(try_word)

'pos'

In [147]:
try_rev = '''''Salman Khans movies are often expected to top in the box offices and thus his movies are often raised to high expectations.This sequel of Ek Tha Tiger, was one of the most anticipated movies in 2017 which has turned out to be quite successful in the box office.The slander this movie receives is unjustified because the movie has done particularly well in terms of plot development and storyline.There are a good amount of plot twists and turns that engage the audience till the end of the story and despite knowing the entire plan of the story, you wait till the end to see how the plan unfolds. The supporting cast have acted particularly well compared to Salman and Katrina. There are a lot of references to the prequel of the story to add slight bits of humor and the action was close to Hollywood level. The antagonist in the story was quite realistic as the movie also serves as an international message showing the reality of war as well as the attempt to kill the stereotype of Pakistan and terrorism.However, this movie was quite predictable and overstretched in terms of the running time. This movie is a good family entertainer and a movie that should be on your watch list.'''''

In [148]:
try_word = word_tokenize(try_rev)

In [150]:
try_word = create_word_features(try_word)

In [151]:
classifier.classify(try_word)

'pos'